In [13]:
!pip install soundfile

In [21]:
import pandas as pd
from pathlib import Path
import soundfile as sf
import os
from tqdm.auto import tqdm

In [22]:
type = 'test'
unique_videos_to_include = 10

In [23]:
df = pd.read_csv(f'{type}.csv')
total_unique_videos = df['parent_name'].unique()

if len(total_unique_videos) > unique_videos_to_include:
    df = df[df['parent_name'].isin(total_unique_videos[:unique_videos_to_include])]

In [24]:
df.head()

,segments_start,segments_end,sentence,chunk_name,parent_name,paths
0,0.0,8.0,लिबर ऑफिस impress में एक प्रस्तुति document बन...,103085_w5Jyq3XMbb3WwiKQ_0000,w5Jyq3XMbb3WwiKQ,data/test_segments/103085_w5Jyq3XMbb3WwiKQ_000...
1,8.0,21.0,इस tutorial में हम impress window के भागों के ...,103085_w5Jyq3XMbb3WwiKQ_0001,w5Jyq3XMbb3WwiKQ,data/test_segments/103085_w5Jyq3XMbb3WwiKQ_000...
2,21.0,29.0,यहाँ हम अपने ऑपरेटिंग सिस्टम के रूप में gnu/li...,103085_w5Jyq3XMbb3WwiKQ_0002,w5Jyq3XMbb3WwiKQ,data/test_segments/103085_w5Jyq3XMbb3WwiKQ_000...
3,29.0,35.0,चलिए अपनी प्रस्तुति प्रेजैटेशन sample impress ...,103085_w5Jyq3XMbb3WwiKQ_0003,w5Jyq3XMbb3WwiKQ,data/test_segments/103085_w5Jyq3XMbb3WwiKQ_000...
4,35.0,39.0,चलिए देखते हैं कि screen पर क्या क्या है,103085_w5Jyq3XMbb3WwiKQ_0004,w5Jyq3XMbb3WwiKQ,data/test_segments/103085_w5Jyq3XMbb3WwiKQ_000...


In [25]:
df.shape

(869, 6)

In [26]:
audios_file_path = Path(f'data/{type}')
output_directory = f'data/{type}_segments'

os.makedirs(output_directory, exist_ok=True)
records = 0
paths = []

for i, file in tqdm(enumerate(df['parent_name'].unique())):

    file_records = df[df['parent_name'] == file]
    segments_start = file_records['segments_start'].values
    segments_end = file_records['segments_end'].values
    chunk_names = file_records['chunk_name'].values

    audio_segments = []
    # Read the WAV file
    audio, sample_rate = sf.read(audios_file_path/f'{file}.wav')
    # Cut audio into segments
    for i, (start_time, end_time, chunk_name) in enumerate(zip(segments_start, segments_end, chunk_names)):
        # print(start_time, end_time)
        start_frame = int(start_time * sample_rate)
        end_frame = int(end_time * sample_rate)
        segment = audio[start_frame:end_frame]
        segment_filepath = os.path.join(output_directory, f"{chunk_name}.wav")
        # Save the segment as a WAV file
        sf.write(segment_filepath, segment, sample_rate)
        paths.append(str(segment_filepath))
        audio_segments.append(segment)
    records += len(audio_segments)    

0it [00:00, ?it/s]

In [27]:
records

869

In [28]:
df['paths'] = paths

In [29]:
df.to_csv(f'{type}.csv', index=False)

In [30]:
df.columns

Index(['segments_start', 'segments_end', 'sentence', 'chunk_name',
       'parent_name', 'paths'],
      dtype='object')